# main

In [1]:
import json
import requests
from main import flatten
import popelines
import os

In [2]:
def fix_values(value, key, reset_key):
    if key == reset_key:
        new_list = []
        for x in value:
            value[x][f'{reset_key}_id'] = x
            new_list.append(value[x])
        return new_list
    else:
        return value

In [3]:
# read endpoint, returns a json file of the HTTP request
def read_endpoint(endpoint, headers_set, params_set=None):
    try:
        response = requests.get(endpoint, headers=headers_set, params=params_set)
        response_text = json.loads(response.text)
        response.raise_for_status()

    except requests.exceptions.HTTPError as err:
        print(err)

    return response_text

In [4]:
# generate all experiments given a project id
def generate_experiments(project_id, experiment_endpoint, experiment_headers, experiment_params):
    # get all experiments from one project
    j_exp = read_endpoint(endpoint=experiment_endpoint, headers_set=experiment_headers, params_set=experiment_params)

    # loop for all experiments in this project
    experiment_id_list = []
    upload_exp_list = []
    for exp in j_exp:
        exp['project_id'] = project_id
        upload_exp_list.append(flatten(exp, {}, ''))
        experiment_id_list.append(exp['id'])
        
    return upload_exp_list, experiment_id_list

In [5]:
# generate all projects within account
def generate_projects(project_endpoint, project_headers):
    # get all projects
    j_proj = read_endpoint(endpoint=project_endpoint, headers_set=project_headers)

    # store a list of project metadata
    return j_proj

# load all projects, get project_id's

In [6]:
gbq_key = os.environ.get('GOOGLE_ACCOUNT_CREDENTIALS')
pope = popelines.popeline(dataset_id='optimizely', service_key_file_loc=gbq_key, directory='.', verbose=False)

# Optimizely parameters
headers = {
    'Authorization': 'Bearer 2:EWAWmaXb4TgtYVU2VvwoEF-9UbJxBahkiFh1633_Oc9nmju7iJis',
}

In [7]:
project_endpoint = 'https://api.optimizely.com/v2/projects'

In [8]:
all_projects = generate_projects(project_endpoint, headers)

In [9]:
pope.write_to_json(file_name='../uploads/projects.json', jayson=all_projects, mode='w')

In [10]:
pope.write_to_bq(table_name='projects', file_name='../uploads/projects.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)
print("Successfully uploaded all projects.")

INFO:root:Writing projects to BQ from file ../uploads/projects.json


2019-01-31 22:08:21,258 - root - Writing projects to BQ from file ../uploads/projects.json
Successfully uploaded all projects.


In [11]:
project_id_list = []
for project in all_projects:
    project_id_list.append((project['id']))

In [12]:
project_id_list


[9965963792, 10347872424, 10427612860, 10429921250, 10849384554]

In [ ]:
project_id_name = []
for project in all_projects:
    project_id_name.append((project['id'], project['name']))

In [ ]:
project_id_name

# load experiments

In [13]:
experiment_endpoint = 'https://api.optimizely.com/v2/experiments'


In [16]:
experiment_id_list = []

for project_id in project_id_list:
    # params include project_id (required) and experiments pulling per each request (default only 25)
    params = (
        ('project_id', project_id),
        ('per_page', 100),
    ) 

    exp_list, exp_id_list = generate_experiments(project_id, experiment_endpoint, headers, params)
    experiment_id_list.extend(exp_id_list)
    
    # upload experiments
    pope.write_to_json(file_name='../uploads/experiments.json', jayson=exp_list, mode='w')
    pope.write_to_bq(table_name='experiments', file_name='../uploads/experiments.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)
    print(f"Successfully uploaded experiments for project {project_id}")

INFO:root:Writing experiments to BQ from file ../uploads/experiments.json


2019-01-31 22:08:59,238 - root - Writing experiments to BQ from file ../uploads/experiments.json
Successfully uploaded experiments for project 9965963792


INFO:root:Writing experiments to BQ from file ../uploads/experiments.json


2019-01-31 22:09:06,461 - root - Writing experiments to BQ from file ../uploads/experiments.json
Successfully uploaded experiments for project 10347872424


INFO:root:Writing experiments to BQ from file ../uploads/experiments.json


2019-01-31 22:09:13,348 - root - Writing experiments to BQ from file ../uploads/experiments.json
Successfully uploaded experiments for project 10427612860


INFO:root:Writing experiments to BQ from file ../uploads/experiments.json


2019-01-31 22:09:20,124 - root - Writing experiments to BQ from file ../uploads/experiments.json
Successfully uploaded experiments for project 10429921250


INFO:root:Writing experiments to BQ from file ../uploads/experiments.json


2019-01-31 22:09:25,139 - root - Writing experiments to BQ from file ../uploads/experiments.json
Successfully uploaded experiments for project 10849384554


# results timeseries

In [17]:
len(experiment_id_list)

108

In [19]:
result_endpoint = f'https://api.optimizely.com/v2/experiments/{experiment_id}/timeseries'

In [ ]:
for experiment_id in experiment_id_list:
        response_ts = requests.get(result_endpoint, headers=headers)

        # if '' then the experiment has not started yet
        if response_ts.text == '':
            j_ts = {'experiment_id': experiment_id}
        else:
            j_ts = json.loads(response_ts.text)

        new_j_ts = pope.fix_json_values(callback=fix_values, obj=j_ts, reset_key='results')

        flattened_metrics = []
        for metric in new_j_ts['metrics']:
            for result in metric['results']:
                for timeseries in result['timeseries']:
                    new_dict = {}
                    new_dict['aggregator'] = metric['aggregator']
                    new_dict['event_id'] = metric['event_id']
                    new_dict['name'] = metric['name']
                    new_dict['scope'] = metric['scope']
                    new_dict['winning_direction'] = metric['winning_direction']

                    new_dict['result_is_baseline'] = result['is_baseline']
                    new_dict['result_level'] = result['level']
                    new_dict['result_name'] = result['name']
                    new_dict['result_variation_id'] = result['variation_id']
                    new_dict['result_id'] = result['results_id']
                    
                    new_dict['time_series_rate'] = timeseries['rate']
                    new_dict['time_series_time'] = timeseries['time']
                    new_dict['time_series_value'] = timeseries['value']
                    new_dict['time_series_samples'] = timeseries['samples']
                    new_dict['time_series_variance'] = timeseries['variance']

                    flattened_metrics.append(new_dict)
        new_j_ts['metrics'] = flattened_metrics

        pope.write_to_json(file_name='../uploads/origin_results.json', jayson=[new_j_ts], mode='w')
        pope.write_to_bq(table_name='results', file_name='../uploads/origin_results.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)
        print(f"Successfully uploaded result time series for experiment {experiment_id}")
        

INFO:root:Writing results to BQ from file ../uploads/origin_results.json


2019-01-31 22:09:45,176 - root - Writing results to BQ from file ../uploads/origin_results.json
Successfully uploaded result time series for experiment 10756790433


INFO:root:Writing results to BQ from file ../uploads/origin_results.json


2019-01-31 22:09:59,715 - root - Writing results to BQ from file ../uploads/origin_results.json
Successfully uploaded result time series for experiment 10796678057


INFO:root:Writing results to BQ from file ../uploads/origin_results.json


...

2019-01-31 22:11:18,979 - root - Writing results to BQ from file ../uploads/origin_results.json
Successfully uploaded result time series for experiment 10556110251


INFO:root:Writing results to BQ from file ../uploads/origin_results.json


2019-01-31 22:11:26,858 - root - Writing results to BQ from file ../uploads/origin_results.json
Successfully uploaded result time series for experiment 10564323775


INFO:root:Writing results to BQ from file ../uploads/origin_results.json


2019-01-31 22:11:32,700 - root - Writing results to BQ from file ../uploads/origin_results.json
